# Understanding Shapley value explanation algorithms for trees

https://hughchen.github.io/its_blog/index.html

# Baseline Shapley value

Shown to be a unique solution to attribution methods by 

```
The many Shapley values for model explanation
M. Sundararajan, A. Najmi.
arXiv preprint arXiv:1908.08474. 2019.
```

* $N$: set of all features.
* $S$ a subset of $N$.
* $x^f$: foreground instance.
* $x^b$: background instance.
* $x^S$: hybrid instance, a function of $x^f$, $x^b$, and $S$, specifically, for each feature, if $i \in S$, $x_i^S = x_i^f$, else $x_i^S = x_i^b$.

The hybrid instance is defined as for each feature $i$, , so it is a function of $x^f$, $x^b$, and $S$, i.e.

$$x^S = h(x^f, x^b, S)$$

$h$ stands for hybrid.

Then the Shapley value for feature $i$ can be calculated as

\begin{align*}
\phi_i(N, f, x^f, x^b)
&= \sum _{S \subseteq N \backslash \{i\}} \frac{s!(n - s - 1)!}{n!} \Big(f \left(x^{S \cup \{i\}} \right) - f \left( x^S \right) \Big) \\
\end{align*}

Note,

* $f$ corresponds to the value function in the canonical Shapley value definition.

We can average over background instances first to obtain $x^b$ or take each instance as a $x^b$ and average over the resulting Shapley values.

# Zero baseline

No good beacuse the meaning of zero is arbitrary in different models, it does not necessarily mean a feature is misisng.

# Average baseline

Difficult to interpret in the case of non-linear models (Not sure what exactly this means)?

> Comparing to the average individual is even more unappealing for non-linear models where comparing to an average individual is not equivalent to comparing to the average population.

# Condtional expectation

# Interventional condtional expectation

Causual inference's interventional conditional expectation:

\begin{align*} 
v(S) = \mathbb{E}_D[f(x) | \text{do}(x_S)]
\end{align*}

Unclear what this equation exactly means, may just refer to causal inference's ICE in general.

Definition of importance attribution for feature $i$ given background data $D$,

\begin{align*}
\psi_i(N, f, x^f, D) 
&= \frac{1}{|D|} \sum_{x^b \in D} \phi_i(N, f, x^f, x^b) \\
\end{align*}

$\phi_i(N, f, x^f, \mathbb{E}[x^b])$ v.s. $\mathbb{E}_D[\phi_i(N, f, x^f, x^b)]$.

from "A Unified Approach to Interpreting Model Predictions"

https://www.mendeley.com/reference-manager/reader/05be0e56-c1b0-30d9-8712-88652f492bf9/7afdb7dd-2eed-ce11-1775-4d0bf0d4c372


>SHAP values are Shapley values of the a conditional expectation function of the original model.

Reproduce calculations in Figure 3 for two baselines. We'll replace $x_1$, $x_2$, $x_3$ with a, b, c for clarity.

In [1]:
import dataclasses
from typing import Dict, Tuple

import shap_demo

get_ipython().magic("load_ext autoreload")
get_ipython().magic("autoreload 2")

Reproduce calculations in Figure 3 for two baselines. We'll replace $x_1$, $x_2$, $x_3$ with a, b, c for clarity.

In [5]:
def linear_model(a, b, c) -> float:
    β1, β2, β3 = 2, -1, 10
    return β1 * a + β2 * b + β3 * c

In [6]:
def make_vals_dict(
    foreground_vals: Tuple[float, float, float],
    background_vals: Tuple[float, float, float],
) -> Dict[str, float]:
    """Makes values dictionary for all subsets."""
    af, bf, cf = foreground_vals
    ab, bb, cb = background_vals
    return {
        "": linear_model(ab, bb, cb),
        "a": linear_model(af, bb, cb),
        "b": linear_model(ab, bf, cb),
        "c": linear_model(ab, bb, cf),
        "ab": linear_model(af, bf, cb),
        "ac": linear_model(af, bb, cf),
        "bc": linear_model(ab, bf, cf),
        "abc": linear_model(af, bf, cf),
    }

### Zero baseline

In [7]:
# f means foreground
af, bf, cf = 70, 135, 0
# b means background
ab, bb, cb = 0, 0, 0

In [8]:
# value function based on a zero baseline.
vals_zero_baseline = make_vals_dict((af, bf, cf), (ab, bb, cb))

In [9]:
ϕ_a, ϕ_b, ϕ_c = shap_demo.calc_shapley_vals(vals_zero_baseline)

In [10]:
ϕ_a, ϕ_b, ϕ_c

(140.0, -135.0, 0.0)

In [11]:
np.testing.assert_allclose(ϕ_a, 140)
np.testing.assert_allclose(ϕ_b, -135)
np.testing.assert_allclose(ϕ_c, 0)

### Average baseline

In [12]:
# f means foreground
af, bf, cf = 70, 135, 0
# b means background
ab, bb, cb = 70, 135, 0.5

In [13]:
vals_avg_baseline = make_vals_dict((af, bf, cf), (ab, bb, cb))

In [14]:
ϕ_a, ϕ_b, ϕ_c = shap_demo.calc_shapley_vals(vals_avg_baseline)

In [15]:
ϕ_a, ϕ_b, ϕ_c

(0.0, 0.0, -5.0)

In [16]:
np.testing.assert_allclose(ϕ_a, 0)
np.testing.assert_allclose(ϕ_b, 0)
np.testing.assert_allclose(ϕ_c, -5)

# Incorporating background distribution